In [8]:
from collections import OrderedDict
from bitarray import bitarray


class RAM:
    def __init__(self):
        self.pages = bitarray(2**36)
        self.pages.setall(0)
        self.empty_pages = 2**36
    def evict_page(self,page):
        self.pages[page]=0
        self.empty_pages += 1
    def has_empty_pages(self):
        return self.empty_pages>0
    def allocate_empty_page(self):
        page = self.pages.index(0)
        self.pages[page]=1
        self.empty_pages -= 1
        return page

        
        
class TLBMiss(Exception):
    pass
class PageFault(Exception):
    pass

class TLB:
    def __init__(self):
        self.processID = None
        self.mapping = OrderedDict()
    def get_entry(self, virtual_page):
        if virtual_page in self.mapping:
            self.mapping.move_to_end(virtual_address)
            return self.mapping[virtual_page]
        else:
            raise TLBMiss(f"TLB miss: virtual page {virtual_page} has no physical page mapping.")
    def add_entry(self, virtual_page, physical_page):
        self.mapping[virtual_page] = physical_page
        # If the TLB has more than 10 entries, remove the least recently used one
        if len(self.mapping) > 10:
            self.mapping.popitem(last=False)
    def clear_TLB(self):
        self.processID = None
        self.mapping = OrderedDict()


class PTE:
    def __init__(self):
        self.processID = None
        self.mapping = OrderedDict()

    def get_physical_page(self, virtual_page):
        if virtual_page not in self.mapping:
            # If the virtual page is not in the mapping, generate a page fault
            raise PageFault(f"Page fault: virtual page {virtual_page} does not exist.")
        elif self.mapping[virtual_page] is None:
            # If the physical page is None, generate a page fault
            raise PageFault(f"Page fault: virtual page {virtual_page} has no physical page mapping.")
        else:
            # If the mapping exists, move the virtual page to the end so that the first element denotes the LRU entry
            self.mapping.move_to_end(virtual_page)
            physical_page = self.mapping[virtual_page]
            return physical_page

    def evict_lru_page(self):
        # Remove the least recently used virtual page and return its mapping
        virtual_page, physical_page = self.mapping.popitem(last=False)
        return (virtual_page, physical_page)

    def add_mapping(self, virtual_page, physical_page):
        # Add a virtual-to-physical page mapping to the mapping dictionary
        self.mapping[virtual_page] = physical_page

class MMU:
    def __init__(self):
        self.pid_pte_mapping = {}
        self.tlb = TLB()
        self.RAM = RAM()  # 2^36 pages in RAM, each page is 2^12 bytes -- Total 48 bits
        self.tlbMisses = 0

    def initialize_page_table_for_process(self, processID):
        # Initialize a new PTE table for the given processID and add it to the mapping
        self.pid_pte_mapping[processID] = PTE()
        self.pid_pte_mapping[processID].processID = processID

    def clear_TLB(self):
        # Clear the TLB by reinitializing it
        self.tlb.clear_TLB()

    def get_physical_address(self, virtual_address, processId):
        if processId not in self.pid_pte_mapping:
            self.initialize_page_table_for_process(processId)
        virtual_page = virtual_address >> 12
        # Check if the physical address is in the TLB
        try:
            physical_page = self.tlb.get_entry(virtual_page)
        except TLBMiss as e:
            self.tlbMisses += 1
            try:
                # Look up the PTE entry for the processID
                pte_entry = self.pid_pte_mapping[processId]

                physical_page = pte_entry.get_physical_page(virtual_page)
            except PageFault as e:
                physical_page = self.get_free_page()
                self.add_pte_mapping(processId, virtual_page, physical_page)
            # Update the TLB
            self.tlb.add_entry(virtual_page,physical_page)
        
        offset = virtual_address & 0xFFF
        physical_address = (physical_page << 12) | offset
        return physical_address

    def get_free_page(self):
        # Get a free physical page from the unused_pages set, or evict an LRU page
        if self.RAM.has_empty_pages():
            physical_page = self.RAM.allocate_empty_page()
        else:
            # pick a process with maximum pages in PTE entry and evict it to harddisk           
            processID, virtual_page = max(self.pid_pte_mapping.items(), key=lambda x: len(x[1].mapping))
            virtual_page, physical_page = self.pid_pte_mapping[processID].evict_lru_page()
        return physical_page

    def add_pte_mapping(self, processID, virtual_page, physical_page):
        # Add a virtual-to-physical page mapping to the PTE entry for the given processID
        pte_entry = self.pid_pte_mapping[processID]
        pte_entry.mapping[virtual_page] = physical_page


In [9]:
import random

def main():
    processes = [1, 2, 3, 4, 5]
    virtual_address_size = 2**48
    mmu = MMU()
    previousProcessId = 0
    for i in range(100):
        processId = random.choice(processes)
        virtual_address = random.randint(0, virtual_address_size - 1)
        
        if previousProcessId is not processId:
            mmu.clear_TLB()
        physical_address = mmu.get_physical_address(virtual_address, processId)
#         print(f"{processId},{virtual_address},{physical_address}")
        previousProcessId = processId
    print("TLB_Misses",mmu.tlbMisses)
if __name__ == "__main__":
    main()


TLB_Misses 100
